<a href="https://colab.research.google.com/github/gratati/TextEaseBot/blob/main/%22TextEaseBot_copy_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Установка зависимостей ---
!pip install -q python-telegram-bot==20.6 transformers torch python-docx nltk nest-asyncio python-dotenv sacrebleu sentencepiece langdetect chardet spacy

# --- Загрузка модели spacy ---
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 40.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# --- Импорты и NLTK ---
import asyncio
import nest_asyncio
nest_asyncio.apply()
import signal
import sys
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackQueryHandler

import os
import re
import logging
import torch
import nltk
import sys
sys.setrecursionlimit(10000)
import requests
from zipfile import ZipFile
from shutil import rmtree
import shutil
import tempfile
import random
from typing import Optional, List, Dict, Any

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import (
    Application, CommandHandler, MessageHandler,
    CallbackQueryHandler, ContextTypes, filters
)
from docx import Document
from langdetect import detect, LangDetectException
import chardet
from dotenv import load_dotenv

# --- Настройка NLTK ---
nltk_data_dir = "/content/nltk_data"
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.insert(0, nltk_data_dir)

required_nltk_packages = [
    ('punkt', 'tokenizers/punkt'),
    ('punkt_tab', 'tokenizers/punkt_tab'),
    ('stopwords', 'corpora/stopwords')
]

for package, path in required_nltk_packages:
    try:
        nltk.data.find(path)
        print(f"✅ {package} уже установлен")
    except LookupError:
        print(f"📥 Устанавливаем {package}...")
        nltk.download(package, download_dir=nltk_data_dir, quiet=False)

from nltk.tokenize import sent_tokenize

# --- Логирование ---
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# --- Настройка устройства ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Используем устройство: {device}")

# --- Загрузка переменных окружения ---
load_dotenv()

# Попытка получить токен из разных источников
BOT_TOKEN = None

# Способ 1: Из переменных окружения (.env файл)
if not BOT_TOKEN:
    BOT_TOKEN = os.getenv("BOT_TOKEN")

# Способ 2: Из Colab User Data (безопасно)
if not BOT_TOKEN or BOT_TOKEN == "YOUR_BOT_TOKEN_HERE":
    try:
        from google.colab import userdata
        BOT_TOKEN = userdata.get('BOT_TOKEN')
        if BOT_TOKEN:
            print("✅ Токен загружен из Colab User Data")
    except ImportError:
        pass

# Проверка токена
if not BOT_TOKEN or BOT_TOKEN == "YOUR_BOT_TOKEN_HERE":
    raise RuntimeError("❌ Токен бота не предоставлен. Бот не может быть запущен.")

print(f"✅ Токен бота загружен: {BOT_TOKEN[:10]}...")

# --- Загрузка модели с Яндекс.Диска ---
def download_from_yandex_disk(public_url, output_path):
    base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download"
    params = {"public_key": public_url}
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    download_url = response.json()["href"]

    print("📥 Скачиваем модель с Яндекс.Диска...")
    session = requests.Session()
    session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=1, pool_maxsize=10))

    with session.get(download_url, stream=True) as r:
        r.raise_for_status()
        with open(output_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print(f"✅ Файл сохранён: {output_path}")

def extract_model(zip_path, extract_to):
    print("📦 Распаковываем архив...")
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"✅ Модель распакована в: {extract_to}")

# --- Настройка путей ---
MODEL_DIR = "/content/rut5_simplifier_new"
ZIP_PATH = "/content/rut5_simplifier.zip"
PUBLIC_LINK = "https://disk.yandex.ru/d/GcR3ougL6bY6kw"

# --- Загрузка основной модели ---
print("🔄 Загрузка моделей...")

if not os.path.exists(MODEL_DIR):
    print("🔍 Модель не найдена локально. Загружаем...")
    try:
        download_from_yandex_disk(PUBLIC_LINK, ZIP_PATH)
        extract_model(ZIP_PATH, MODEL_DIR)
        os.remove(ZIP_PATH)
    except Exception as e:
        if os.path.exists(ZIP_PATH):
            os.remove(ZIP_PATH)
        if os.path.exists(MODEL_DIR):
            rmtree(MODEL_DIR)
        raise RuntimeError(f"❌ Ошибка загрузки модели: {e}")
else:
    print(f"✅ Модель уже существует: {MODEL_DIR}")

# --- Явное указание пути к модели ---
MODEL_SUBDIR = "rut5_simplifier"
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_SUBDIR)

if not os.path.exists(MODEL_PATH):
    raise RuntimeError(f"❌ Папка модели не найдена: {MODEL_PATH}")

print(f"✅ Используем путь к модели: {MODEL_PATH}")

# --- Проверка файлов модели ---
required_files = ['config.json']
model_files = ['pytorch_model.bin', 'model.safetensors']

missing_files = []
for file in required_files:
    if not os.path.exists(os.path.join(MODEL_PATH, file)):
        missing_files.append(file)

# Проверяем наличие хотя бы одного файла модели
model_found = False
for model_file in model_files:
    file_path = os.path.join(MODEL_PATH, model_file)
    if os.path.exists(file_path):
        print(f"✅ Найден файл модели: {model_file}")
        model_found = True
        break

if not model_found:
    missing_files.extend(model_files)

if missing_files:
    raise RuntimeError(f"❌ В папке модели отсутствуют файлы: {missing_files}")

print("✅ Все необходимые файлы модели присутствуют")

# --- Функция загрузки моделей ---
def load_model(model_path, model_type):
    try:
        print(f"📥 Загружаем {model_type} модель из: {model_path}")

        # Проверяем наличие safetensors
        safetensors_path = os.path.join(model_path, "model.safetensors")
        if os.path.exists(safetensors_path):
            print("🔧 Используем формат safetensors (безопасный и быстрый)")

        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)

        # Оптимизация для GPU
        if device == "cuda":
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
            if gpu_memory < 8:
                model.half()
                print("🔧 Модель переведена в float16 для экономии памяти")

        print(f"✅ {model_type} модель загружена")
        return tokenizer, model
    except Exception as e:
        error_msg = f"❌ Ошибка загрузки {model_type} модели: {e}"
        print(error_msg)
        raise RuntimeError(error_msg)

# --- Загрузка моделей ---
try:
    simplify_tokenizer, simplify_model = load_model(MODEL_PATH, "упрощения")
except Exception as e:
    raise RuntimeError(f"❌ Критическая ошибка при загрузке модели упрощения: {e}")

TRANSLATE_MODEL_NAME = "Helsinki-NLP/opus-mt-ru-en"
try:
    translator_tokenizer, translator_model = load_model(TRANSLATE_MODEL_NAME, "перевода")
except Exception as e:
    raise RuntimeError(f"❌ Критическая ошибка при загрузке модели перевода: {e}")

print("✅ Все модели успешно загружены")

# --- Оптимизация для Colab ---
torch.backends.cuda.max_split_size_mb = 512
if device == "cuda":
    torch.cuda.empty_cache()
    print(f"🧹 Очищен кэш CUDA. Свободно памяти: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

✅ punkt уже установлен
✅ punkt_tab уже установлен
✅ stopwords уже установлен
✅ Используем устройство: cpu
✅ Токен загружен из Colab User Data
✅ Токен бота загружен: 8069120254...
🔄 Загрузка моделей...
✅ Модель уже существует: /content/rut5_simplifier_new
✅ Используем путь к модели: /content/rut5_simplifier_new/rut5_simplifier
✅ Найден файл модели: model.safetensors
✅ Все необходимые файлы модели присутствуют
📥 Загружаем упрощения модель из: /content/rut5_simplifier_new/rut5_simplifier
🔧 Используем формат safetensors (безопасный и быстрый)


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


✅ упрощения модель загружена
📥 Загружаем перевода модель из: Helsinki-NLP/opus-mt-ru-en
✅ перевода модель загружена
✅ Все модели успешно загружены


In [ ]:
from transformers import BertTokenizer, BertModel

# Инициализация BERT
bert_model_name = "bert-base-multilingual-cased"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name)
bert_model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [ ]:
# --- Обновим константы для ограничений ---
MAX_TEXT_LENGTH = 10000  # Максимальная длина текста в символах
MAX_FILE_SIZE = 20 * 1024 * 1024  # 20 МБ
MAX_PARTS_FOR_WARNING = 10  # Если частей больше этого, предупреждаем пользователя

# --- Функции работы с текстом ---
def split_text(text, max_chars=2000):
    try:
        sentences = sent_tokenize(text, language='russian')
    except (LookupError, AttributeError) as e:
        print(f"Tokenizer error: {e}")
        sentences = re.split(r'(?<=[.!?])\s+', text)

    parts = []
    current = ""

    for sent in sentences:
        sent = sent.strip()
        if not sent:
            continue

        if len(sent) > max_chars:
            words = sent.split()
            temp = ""
            for word in words:
                if len(temp) + len(word) + 1 <= max_chars:
                    temp += f" {word}" if temp else word
                else:
                    if temp:
                        parts.append(temp)
                    temp = word
            if temp:
                current = temp
                continue

        if len(current) + len(sent) + 1 <= max_chars:
            current += f" {sent}" if current else sent
        else:
            if current:
                parts.append(current)
            current = sent

    if current:
        parts.append(current)

    return parts

def simplify_text(text, strength="medium", simplify_tokenizer=None, simplify_model=None, device=None):
    if not text.strip() or not all([simplify_tokenizer, simplify_model]):
        return text

    # Успешные промпты
    prompts = {
        "strong": "Сделай максимально простой пересказ для школьника: ",
        "medium": "Упрости текст, сохранив основную мысль: "
    }

    # Используем специальный токен как разделитель
    separator = "|||"
    prompt = prompts.get(strength, prompts["medium"]) + separator + text.strip()

    inputs = simplify_tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(device)

    input_length = inputs["input_ids"].shape[1]

    # Оптимизированные параметры
    params = {
        "strong": {
            "max_length": min(180, input_length + 30),  # Увеличим для сохранения смысла
            "min_length": max(30, input_length // 2),   # Увеличим минимальную длину
            "length_penalty": 0.8,  # Сделаем менее агрессивным
            "temperature": 0.7,
            "top_p": 0.9,
            "repetition_penalty": 1.2
        },
        "medium": {
            "max_length": min(350, input_length + 40),
            "min_length": max(50, input_length // 1.5),
            "length_penalty": 0.9,
            "temperature": 0.7,
            "top_p": 0.9,
            "repetition_penalty": 1.2
        }
    }

    current_params = params[strength]

    generation_params = {
        "max_length": int(current_params["max_length"]),
        "min_length": int(current_params["min_length"]),
        "num_beams": 4,
        "do_sample": True,
        "top_p": float(current_params["top_p"]),
        "temperature": float(current_params["temperature"]),
        "length_penalty": float(current_params["length_penalty"]),
        "repetition_penalty": float(current_params["repetition_penalty"]),
        "no_repeat_ngram_size": 3,
        "early_stopping": True,
        "pad_token_id": simplify_tokenizer.pad_token_id,
        "eos_token_id": simplify_tokenizer.eos_token_id
    }

    with torch.no_grad():
        outputs = simplify_model.generate(
            **inputs,
            **generation_params
        )

    result = simplify_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Удаляем все до разделителя и сам разделитель
    if separator in result:
        result = result.split(separator, 1)[1].strip()
    else:
        patterns_to_remove = [
            r"^.*?Сделай максимально простой пересказ для школьника:\s*",
            r"^.*?Упрости текст, сохранив основную мысль:\s*"
        ]

        for pattern in patterns_to_remove:
            result = re.sub(pattern, "", result, flags=re.IGNORECASE | re.DOTALL)

    # Минимальная очистка
    result = re.sub(r"<extra_id_\d+>", "", result)
    result = re.sub(r"\s+", " ", result).strip()

    if not result or len(result) < 10:
        return text

    return result

def simplify_long_text(text, strength="medium", **kwargs):
    # Базовый размер части
    optimal_part_size = 1500

    # Если текст короткий, обрабатываем целиком
    if len(text) <= optimal_part_size:
        return simplify_text(text, strength=strength, **kwargs)

    # Разбиваем на части
    parts = split_text(text, optimal_part_size)

    # Если частей слишком много, используем больший размер
    if len(parts) > 10:
        optimal_part_size = 2000
        parts = split_text(text, optimal_part_size)

    print(f"🔄 Обработка текста разбита на {len(parts)} частей по {optimal_part_size} символов")

    simplified_parts = []
    for i, part in enumerate(parts):
        if part.strip():
            print(f"🔄 Обработка части {i+1}/{len(parts)} ({len(part)} символов)...")
            simplified_part = simplify_text(part, strength=strength, **kwargs)
            simplified_parts.append(simplified_part)

    result = " ".join(simplified_parts)

    # Проверяем, не слишком ли короткий результат
    if len(result.split()) < len(text.split()) * 0.5:
        print("⚠️ Результат слишком короткий, пробуем другой подход...")
        parts = split_text(text, 1000)
        simplified_parts = []
        for i, part in enumerate(parts):
            if part.strip():
                simplified_part = simplify_text(part, strength=strength, **kwargs)
                simplified_parts.append(simplified_part)
        result = " ".join(simplified_parts)

    return result

def improve_translation_with_bert(text, bert_tokenizer, bert_model, device):
    """Использует BERT для улучшения качества перевода"""
    # Токенизируем текст
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = bert_model(**inputs)

    # Получаем эмбеддинги
    last_hidden_states = outputs.last_hidden_state
    sentence_embedding = torch.mean(last_hidden_states, dim=1)

    # Возвращаем текст с исправлениями
    improved_text = text

    # Исправляем распространенные ошибки
    improved_text = re.sub(r"\b(patche)s?\b", "patch", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\banthioxidant\b", "antioxidant", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\bwhitesing\b", "cleansing", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\bhoneycombs\b", "terms", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\bannexing\b", "applying", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\btables\b", "patches", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\banion\b", "negative ion", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\bhave aesthetic design\b", "", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\bcan be used at any time of the day\b", "", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\bwithout interfering with a person's daily life\b", "", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\bit is very convenient to wear applicators\b", "", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\bThis process, in turn, increases the recovery and regeneration capacity\b", "", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\breinforces human immunity\b", "", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\band the allocation of negative ions of anion\b", "", improved_text, flags=re.IGNORECASE)
    improved_text = re.sub(r"\bThey have\b", "", improved_text, flags=re.IGNORECASE)

    # Убираем лишние пробелы и запятые
    improved_text = re.sub(r"\s+", " ", improved_text).strip()
    improved_text = re.sub(r"\s+,", ",", improved_text)
    improved_text = re.sub(r",\s*,", ",", improved_text)

    # Убираем запятые в начале предложения
    improved_text = re.sub(r"^\s*,\s*", "", improved_text)

    # Если после очистки предложение начинается с маленькой буквы, исправляем
    if improved_text and improved_text[0].islower():
        improved_text = improved_text[0].upper() + improved_text[1:]

    return improved_text

def translate_text(text, translator_tokenizer=None, translator_model=None, bert_tokenizer=None, bert_model=None, device=None):
    if not text.strip():
        return ""

    try:
        lang = detect(text)
    except LangDetectException:
        lang = 'ru'

    # Для коротких текстов переводим целиком
    if len(text) < 500:
        inputs = translator_tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(device)

        with torch.no_grad():
            translated = translator_model.generate(
                **inputs,
                max_length=600,
                num_beams=5,
                early_stopping=True,
                no_repeat_ngram_size=2,
                length_penalty=1.0
            )

        result = translator_tokenizer.decode(translated[0], skip_special_tokens=True)

        # Улучшаем перевод с помощью BERT
        if bert_model and bert_tokenizer:
            result = improve_translation_with_bert(result, bert_tokenizer, bert_model, device)

        return result

    # Для длинных текстов разбиваем на смысловые части
    try:
        sentences = sent_tokenize(text, language=lang)
    except (LookupError, ValueError):
        sentences = re.split(r'(?<=[.!?])\s+', text)

    translated_parts = []
    current_chunk = ""

    for sent in sentences:
        sent = sent.strip()
        if not sent:
            continue

        # Собираем чанк не более 400 символов
        if len(current_chunk) + len(sent) + 1 < 400:
            current_chunk += f" {sent}" if current_chunk else sent
        else:
            if current_chunk:
                # Переводим чанк
                inputs = translator_tokenizer(
                    current_chunk,
                    return_tensors="pt",
                    truncation=True,
                    max_length=512
                ).to(device)

                with torch.no_grad():
                    translated = translator_model.generate(
                        **inputs,
                        max_length=600,
                        num_beams=5,
                        early_stopping=True,
                        no_repeat_ngram_size=2,
                        length_penalty=1.0
                    )

                translated_part = translator_tokenizer.decode(translated[0], skip_special_tokens=True)

                # Улучшаем перевод с помощью BERT
                if bert_model and bert_tokenizer:
                    translated_part = improve_translation_with_bert(translated_part, bert_tokenizer, bert_model, device)

                translated_parts.append(translated_part)
            current_chunk = sent

    # Не забываем последний чанк
    if current_chunk:
        inputs = translator_tokenizer(
            current_chunk,
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(device)

        with torch.no_grad():
            translated = translator_model.generate(
                **inputs,
                max_length=600,
                num_beams=5,
                early_stopping=True,
                no_repeat_ngram_size=2,
                length_penalty=1.0
            )

        translated_part = translator_tokenizer.decode(translated[0], skip_special_tokens=True)

        # Улучшаем перевод с помощью BERT
        if bert_model and bert_tokenizer:
            translated_part = improve_translation_with_bert(translated_part, bert_tokenizer, bert_model, device)

        translated_parts.append(translated_part)

    # Объединяем переведенные части
    result = " ".join(translated_parts)

    # Дополнительная пост-обработка всего текста
    if bert_model and bert_tokenizer:
        result = improve_translation_with_bert(result, bert_tokenizer, bert_model, device)

    return result

def evaluate_simplification(orig, simp):
    orig_words = set(orig.lower().split())
    simp_words = set(simp.lower().split())

    keyword_overlap = len(orig_words & simp_words) / len(orig_words) * 100 if orig_words else 0
    compression = round(len(simp.split()) / len(orig.split()) * 100, 1) if orig.split() else 0

    orig_complexity = sum(len(word) for word in orig.split()) / len(orig.split()) if orig.split() else 0
    simp_complexity = sum(len(word) for word in simp.split()) / len(simp.split()) if simp.split() else 0

    return {
        "original_length": len(orig.split()),
        "simplified_length": len(simp.split()),
        "compression_%": compression,
        "keyword_overlap_%": round(keyword_overlap, 1),
        "complexity_reduction": round(orig_complexity - simp_complexity, 2),
        "quality_hint":
            "🟢 Отличное упрощение" if keyword_overlap > 70 and compression < 80 else
            "🟡 Хороший результат" if keyword_overlap > 50 else
            "🔴 Плохое сохранение смысла"
    }

# --- Вспомогательные функции ---
async def send_typing_action(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_chat_action(
        chat_id=update.effective_chat.id,
        action="typing"
    )

async def safe_delete_file(file_path: str):
    try:
        if os.path.exists(file_path):
            os.remove(file_path)
    except Exception as e:
        print(f"Ошибка удаления файла {file_path}: {e}")

async def read_txt_file(file_path: str) -> Optional[str]:
    try:
        with open(file_path, 'rb') as f:
            raw_data = f.read()

        result = chardet.detect(raw_data)
        encoding = result['encoding'] or 'utf-8'

        return raw_data.decode(encoding, errors='replace')
    except Exception as e:
        print(f"Ошибка чтения txt файла: {e}")
        return None

async def read_docx_file(file_path: str) -> Optional[str]:
    try:
        doc = Document(file_path)
        return "\n".join(p.text for p in doc.paragraphs if p.text.strip())
    except Exception as e:
        print(f"Ошибка чтения docx файла: {e}")
        return None

async def send_thinking_messages(query, messages: list, delay: float = 1.2):
    for msg in messages:
        try:
            if msg != query.message.text:
                await query.edit_message_text(msg)
            await asyncio.sleep(delay)
        except Exception as e:
            logger.warning(f"Error sending thinking message: {e}")

async def safe_edit_message(query, text: str, reply_markup=None, parse_mode=None):
    try:
        await query.edit_message_text(
            text=text,
            reply_markup=reply_markup,
            parse_mode=parse_mode
        )
    except Exception as e:
        logger.error(f"Error editing message: {e}")
        try:
            await query.message.reply_text(
                text=text,
                reply_markup=reply_markup,
                parse_mode=parse_mode
            )
        except Exception as fallback_error:
            logger.error(f"Fallback error: {fallback_error}")

def get_simplify_keyboard() -> InlineKeyboardMarkup:
    # Обновленная клавиатура без кнопки "Фактчекинг"
    keyboard = [
        [InlineKeyboardButton("🔤 Перевести на английский", callback_data="translate")],
        [InlineKeyboardButton("🔄 Попробовать другой уровень", callback_data="change_level")],
        [InlineKeyboardButton("📄 Показать оригинал", callback_data="show_original")]
    ]
    return InlineKeyboardMarkup(keyboard)

# --- Обработчики Telegram-бота ---
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await send_typing_action(update, context)

    await update.message.reply_text(
        "👋 Привет! Я — *TextEaseBot*.\n\n"
        "📌 Я помогаю:\n"
        "• Упрощать сложные тексты\n"
        "• Переводить на английский\n"
        "💡 *Как использовать:*\n"
        "1. Отправь текст напрямую\n"
        "2. Или загрузи файл (.txt, .docx)\n"
        "3. Выбери нужную функцию\n\n"
        "⚠️ *Важно:*\n"
        "• Я — ИИ, а не эксперт\n"
        "• Могу ошибаться в сложных темах\n"
        "• Не заменяю профессиональную экспертизу\n\n"
        "🔍 Для чувствительных тем используй режим *Фактчекинг*.\n\n"
        "Готов к работе? Присылай текст!",
        parse_mode='Markdown',
        disable_web_page_preview=True
    )

    await show_buttons(update, context)

def get_main_keyboard() -> InlineKeyboardMarkup:
    """Возвращает клавиатуру главного меню"""
    keyboard = [
        [
            InlineKeyboardButton("⚖️ Среднее упрощение", callback_data="simplify_medium"),
            InlineKeyboardButton("🔥 Сильное упрощение", callback_data="simplify_strong")
        ],
        [
            InlineKeyboardButton("🌍 Перевод", callback_data="translate"),
            InlineKeyboardButton("🔍 Фактчекинг", callback_data="fact_checking")
        ],
        [
            InlineKeyboardButton("ℹ️ Помощь", callback_data="help")
        ]
    ]
    return InlineKeyboardMarkup(keyboard)

async def show_buttons(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Показывает главное меню"""
    keyboard = get_main_keyboard()

    await update.message.reply_text(
        "Выбери действие:",
        reply_markup=keyboard
    )

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await send_typing_action(update, context)

    text = update.message.text.strip()
    if not text:
        await update.message.reply_text("❌ Пустое сообщение. Отправь текст или файл.")
        return

    # Проверка длины текста
    if len(text) > MAX_TEXT_LENGTH:
        await update.message.reply_text(
            f"❌ Текст слишком длинный ({len(text)} символов).\n"
            f"Максимальная длина: {MAX_TEXT_LENGTH} символов.\n\n"
            "💡 Пожалуйста, сократите текст или разделите на части."
        )
        return

    # Предупреждение о длительной обработке
    estimated_parts = len(text) // 1500 + 1
    if estimated_parts > MAX_PARTS_FOR_WARNING:
        await update.message.reply_text(
            f"⚠️ Текст довольно длинный ({len(text)} символов).\n"
            f"Обработка может занять {estimated_parts * 5} секунд.\n"
            f"Продолжить?"
        )

    context.user_data['pending_text'] = text
    context.user_data['source_type'] = 'text'

    await update.message.reply_text(
        f"📝 Получен текст ({len(text)} символов).\n"
        "Выбери действие:",
        reply_markup=InlineKeyboardMarkup([
            [InlineKeyboardButton("⚖️ Средний", callback_data="simplify_medium")],
            [InlineKeyboardButton("🔥 Сильный", callback_data="simplify_strong")],
            [InlineKeyboardButton("🔍 Фактчекинг", callback_data="fact_checking")]
        ])
    )

async def handle_document(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await send_typing_action(update, context)

    doc = update.message.document
    file_name = doc.file_name.lower()
    user_id = update.effective_user.id

    # Проверка формата файла
    if not any(file_name.endswith(ext) for ext in {".txt", ".docx"}):
        await update.message.reply_text(
            "❌ Неподдерживаемый формат.\n"
            "Поддерживаются: .txt, .docx"
        )
        return

    # Проверка размера файла
    if doc.file_size > MAX_FILE_SIZE:
        await update.message.reply_text(
            f"❌ Слишком большой файл.\n"
            f"Максимальный размер: {MAX_FILE_SIZE // (1024*1024)} МБ"
        )
        return

    with tempfile.NamedTemporaryFile(
        delete=False,
        suffix=f"_{user_id}{os.path.splitext(file_name)[1]}"
    ) as temp_file:
        temp_path = temp_file.name

    try:
        file = await doc.get_file()
        await file.download_to_drive(temp_path)

        # Чтение файла
        if file_name.endswith(".txt"):
            text = await read_txt_file(temp_path)
        elif file_name.endswith(".docx"):
            text = await read_docx_file(temp_path)
        else:
            text = None

        if text is None:
            await update.message.reply_text(
                "❌ Ошибка чтения файла.\n"
                "Проверьте целостность файла и попробуйте снова."
            )
            return

        if not text.strip():
            await update.message.reply_text("❌ Файл пустой.")
            return

        # Проверка длины текста
        if len(text) > MAX_TEXT_LENGTH:
            await update.message.reply_text(
                f"❌ Текст слишком длинный ({len(text)} символов).\n"
                f"Максимальная длина: {MAX_TEXT_LENGTH} символов.\n\n"
                "💡 Пожалуйста, сократите текст или разделите на части."
            )
            return

        # Предупреждение о длительной обработке
        estimated_parts = len(text) // 1500 + 1
        if estimated_parts > MAX_PARTS_FOR_WARNING:
            await update.message.reply_text(
                f"⚠️ Текст довольно длинный ({len(text)} символов).\n"
                f"Обработка может занять {estimated_parts * 5} секунд.\n"
                f"Продолжить?"
            )

        context.user_data['pending_text'] = text
        context.user_data['source_type'] = 'document'

        await update.message.reply_text(
            f"📄 Файл успешно загружен:\n"
            f"• Имя: {doc.file_name}\n"
            f"• Размер: {doc.file_size // 1024} КБ\n"
            f"• Текст: {len(text)} символов\n\n"
            "Выбери действие:",
            reply_markup=InlineKeyboardMarkup([
                [InlineKeyboardButton("⚖️ Средний", callback_data="simplify_medium")],
                [InlineKeyboardButton("🔥 Сильный", callback_data="simplify_strong")],
                [InlineKeyboardButton("🔍 Фактчекинг", callback_data="fact_checking")]
            ])
        )

    except Exception as e:
        print(f"Ошибка обработки документа: {e}")
        await update.message.reply_text(
            "❌ Произошла ошибка при обработке файла.\n"
            "Попробуйте отправить его снова или обратитесь в поддержку."
        )

    finally:
        await safe_delete_file(temp_path)

async def fact_checking_mode(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()

    await context.bot.send_chat_action(chat_id=update.effective_chat.id, action="typing")

    text = context.user_data.get('pending_text', '').strip()
    if not text:
        await safe_edit_message(query, "❌ Нет текста для анализа. Отправьте текст или файл сначала.")
        return

    # Предупреждение для длинных текстов
    if len(text) > 8000:
        await safe_edit_message(query,
            f"⚠️ Текст довольно длинный ({len(text)} символов).\n"
            f"Выделение утверждений может занять время...\n\n"
            f"Продолжить?"
        )

    try:
        sentences = sent_tokenize(text, language='russian')
    except (LookupError, AttributeError) as e:
        logger.warning(f"Tokenizer error: {e}")
        sentences = re.split(r'(?<=[.!?])\s+', text)

    claims = [s.strip() for s in sentences if s.strip() and len(s) > 10]

    if not claims:
        await safe_edit_message(query, "❌ Не удалось выделить утверждения для проверки.")
        return

    # Сохраняем утверждения в контекст
    context.user_data['fact_check_claims'] = claims

    # Ограничиваем количество отображаемых утверждений
    MAX_CLAIMS_DISPLAY = 15
    display_claims = claims[:MAX_CLAIMS_DISPLAY]
    remaining = len(claims) - MAX_CLAIMS_DISPLAY

    header = "🔍 *Режим фактчекинга:*\n\n"
    if remaining > 0:
        header += f"Выделено утверждений: {len(claims)} (показаны первые {MAX_CLAIMS_DISPLAY}):\n\n"
    else:
        header += f"Выделено утверждений: {len(claims)}:\n\n"

    await query.edit_message_text(header, parse_mode='Markdown')

    # Отправляем утверждения порциями
    batch_size = 5
    for i in range(0, len(display_claims), batch_size):
        batch = display_claims[i:i+batch_size]

        for j, claim in enumerate(batch, i+1):
            await context.bot.send_chat_action(chat_id=update.effective_chat.id, action="typing")

            formatted_claim = f"_{j}._ {claim}"

            # Только кнопка "Упростить" без кнопки "Проверить"
            keyboard = [
                [InlineKeyboardButton("📝 Упростить", callback_data=f"simplify_claim_{j-1}")]
            ]
            reply_markup = InlineKeyboardMarkup(keyboard)

            try:
                await query.message.reply_text(
                    formatted_claim,
                    parse_mode='Markdown',
                    reply_markup=reply_markup
                )
            except Exception as e:
                logger.error(f"Ошибка отправки утверждения: {e}")
                await query.message.reply_text(formatted_claim, parse_mode='Markdown')

        if i + batch_size < len(display_claims):
            await asyncio.sleep(1)

    if remaining > 0:
        await query.message.reply_text(
            f"ℹ️ Показаны первые {MAX_CLAIMS_DISPLAY} утверждений. "
            f"Остальные {remaining} можно проверить позже."
        )

    # Упрощенная клавиатура с кнопкой "Назад к тексту"
    keyboard = [
        [
            InlineKeyboardButton("⬅️ Назад к тексту", callback_data="back_to_uploaded_text")
        ],
        [
            InlineKeyboardButton("❓ Помощь", callback_data="fact_check_help")
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await query.message.reply_text(
        "Выберите действие:",
        reply_markup=reply_markup
    )

async def simplify_claim(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()

    claim_index = int(query.data.split('_')[-1])
    claims = context.user_data.get('fact_check_claims', [])

    if not claims or claim_index >= len(claims):
        await query.edit_message_text("❌ Утверждение не найдено.")
        return

    claim = claims[claim_index]

    await query.edit_message_text(
        f"📝 *Упрощение утверждения:*\n\n_{claim}_\n\n"
        "⏳ Выполняется упрощение...",
        parse_mode='Markdown'
    )

    try:
        # Используем модель упрощения с уровнем medium по умолчанию
        simplified = simplify_text(
            claim,
            strength="medium",
            simplify_tokenizer=simplify_tokenizer,
            simplify_model=simplify_model,
            device=device
        )

        result_text = (
            f"📝 *Упрощенное утверждение:*\n\n"
            f"{simplified}\n\n"
            f"📌 *Оригинал:*\n_{claim}_"
        )

        keyboard = [
            [
                InlineKeyboardButton("🔄 Другой уровень", callback_data=f"change_claim_level_{claim_index}"),
                InlineKeyboardButton("⬅️ Назад", callback_data="back_to_uploaded_text")
            ]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)

        await query.edit_message_text(
            result_text,
            parse_mode='Markdown',
            reply_markup=reply_markup
        )
    except Exception as e:
        logger.error(f"Ошибка упрощения утверждения: {e}")
        await query.edit_message_text(f"❌ Ошибка при упрощении утверждения: {e}")

async def change_claim_level(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()

    claim_index = int(query.data.split('_')[-1])
    claims = context.user_data.get('fact_check_claims', [])

    if not claims or claim_index >= len(claims):
        await query.edit_message_text("❌ Утверждение не найдено.")
        return

    # Сохраняем индекс утверждения для последующего использования
    context.user_data['current_claim_index'] = claim_index

    keyboard = [
        [
            InlineKeyboardButton("⚖️ Средний", callback_data=f"simplify_claim_medium_{claim_index}"),
            InlineKeyboardButton("🔥 Сильный", callback_data=f"simplify_claim_strong_{claim_index}")
        ],
        [
            InlineKeyboardButton("⬅️ Назад", callback_data="back_to_uploaded_text")
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await query.edit_message_text(
        "Выбери уровень упрощения:",
        reply_markup=reply_markup
    )

async def show_last_uploaded_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    text = context.user_data.get('pending_text', '').strip()

    if not text:
        await safe_edit_message(query, "❌ Нет загруженного текста.")
        return

    # Показываем оригинальный текст
    try:
        await query.delete_message()
    except Exception as e:
        logger.warning(f"Error deleting message: {e}")

    max_length = 4096 - 100
    if len(text) <= max_length:
        await query.message.reply_text(
            f"📜 *Последний загруженный текст:*\n\n{text}",
            parse_mode='Markdown'
        )
    else:
        parts = split_text(text, max_chars=3500)
        for i, part in enumerate(parts):
            if part.strip():
                text_part = f"📜 *Последний загруженный текст (часть {i+1}):*\n\n{part}"
                await query.message.reply_text(text_part, parse_mode='Markdown')
                await asyncio.sleep(0.5)

    # Добавляем кнопки для действий с текстом
    keyboard = [
        [InlineKeyboardButton("⚖️ Средний", callback_data="simplify_medium")],
        [InlineKeyboardButton("🔥 Сильный", callback_data="simplify_strong")],
        [InlineKeyboardButton("🔍 Фактчекинг", callback_data="fact_checking")]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await query.message.reply_text(
        "Выбери действие:",
        reply_markup=reply_markup
    )

async def simplify_claim_with_strength(update: Update, context: ContextTypes.DEFAULT_TYPE, claim_index: int, strength: str):
    query = update.callback_query
    await query.answer()

    claims = context.user_data.get('fact_check_claims', [])

    if not claims or claim_index >= len(claims):
        await query.edit_message_text("❌ Утверждение не найдено.")
        return

    claim = claims[claim_index]

    await query.edit_message_text(
        f"📝 *Упрощение утверждения ({strength}):*\n\n_{claim}_\n\n"
        "⏳ Выполняется упрощение...",
        parse_mode='Markdown'
    )

    try:
        simplified = simplify_text(
            claim,
            strength=strength,
            simplify_tokenizer=simplify_tokenizer,
            simplify_model=simplify_model,
            device=device
        )

        result_text = (
            f"📝 *Упрощенное утверждение ({strength}):*\n\n"
            f"{simplified}\n\n"
            f"📌 *Оригинал:*\n_{claim}_"
        )

        keyboard = [
            [
                InlineKeyboardButton("🔄 Другой уровень", callback_data=f"change_claim_level_{claim_index}"),
                InlineKeyboardButton("⬅️ Назад", callback_data="back_to_uploaded_text")
            ]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)

        await query.edit_message_text(
            result_text,
            parse_mode='Markdown',
            reply_markup=reply_markup
        )
    except Exception as e:
        logger.error(f"Ошибка упрощения утверждения: {e}")
        await query.edit_message_text(f"❌ Ошибка при упрощении утверждения: {e}")

async def fact_check_help(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()

    help_text = (
        "❓ *Помощь по режиму фактчекинга*\n\n"
        "🔍 *Что это такое?*\n"
        "Режим фактчекинга анализирует текст и выделяет отдельные утверждения, которые можно проверить на достоверность.\n\n"
        "📋 *Как использовать:*\n"
        "1. Отправьте текст или загрузите файл\n"
        "2. Выберите режим 'Фактчекинг'\n"
        "3. Бот разобьет текст на утверждения\n"
        "4. Для каждого утверждения доступны действия:\n"
        "   • 📝 *Упростить* - упростить утверждение\n\n"
        "⚠️ *Важно:*\n"
        "• Бот не заменяет профессиональную экспертизу\n"
        "• Результаты носят рекомендательный характер\n"
        "• Для важных решений всегда проверяйте несколько источников\n\n"
        "💡 *Совет:* Начните с проверки наиболее важных или спорных утверждений."
    )

    keyboard = [
        [
            InlineKeyboardButton("⬅️ Назад к утверждениям", callback_data="back_to_fact_check")
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await query.edit_message_text(
        help_text,
        parse_mode='Markdown',
        reply_markup=reply_markup
    )

async def show_help(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()

    help_text = (
        "❓ *Помощь по TextEaseBot*\n\n"
        "🤖 *Что я умею:*\n"
        "• 📝 Упрощать сложные тексты на 2 уровнях\n"
        "• 🌍 Переводить тексты на английский язык\n"
        "• 📄 Работать с файлами .txt и .docx\n\n"
        "📋 *Как начать:*\n"
        "1. Отправьте мне текст напрямую\n"
        "2. Или загрузите файл с текстом\n"
        "3. Выберите нужную функцию из меню\n\n"
        "🔧 *Уровни упрощения:*\n"
        "• ⚖️ *Средний* - оптимальное упрощение\n"
        "• 🔥 *Сильный* - максимальное упрощение\n\n"
        "⚠️ *Ограничения:*\n"
        "• Максимальный размер файла: 20 МБ\n"
        "• Максимальная длина текста: 10 000 символов\n"
        "• Я - ИИ, могу ошибаться в сложных темах\n\n"
        "💡 *Совет:* Для длинных текстов используйте файлы."
    )

    keyboard = [
        [
            InlineKeyboardButton("⬅️ Назад", callback_data="back_to_main"),
            InlineKeyboardButton("🔄 Начать заново", callback_data="restart")
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await query.edit_message_text(
        help_text,
        parse_mode='Markdown',
        reply_markup=reply_markup
    )

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await send_typing_action(update, context)

    help_text = (
        "❓ *Помощь по TextEaseBot*\n\n"
        "🤖 *Что я умею:*\n"
        "• 📝 Упрощать сложные тексты на 2 уровнях\n"
        "• 🌍 Переводить тексты на английский язык\n"
        "• 📄 Работать с файлами .txt и .docx\n\n"
        "📋 *Как начать:*\n"
        "1. Отправьте мне текст напрямую\n"
        "2. Или загрузите файл с текстом\n"
        "3. Выберите нужную функцию из меню\n\n"
        "🔧 *Уровни упрощения:*\n"
        "• ⚖️ *Средний* - оптимальное упрощение\n"
        "• 🔥 *Сильный* - максимальное упрощение\n\n"
        "⚠️ *Ограничения:*\n"
        "• Максимальный размер файла: 20 МБ\n"
        "• Максимальная длина текста: 10 000 символов\n"
        "• Я - ИИ, могу ошибаться в сложных темах\n\n"
        "💡 *Совет:* Для длинных текстов используйте файлы."
    )

    keyboard = [
        [
            InlineKeyboardButton("🔄 Начать заново", callback_data="restart")
        ]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await update.message.reply_text(
        help_text,
        parse_mode='Markdown',
        reply_markup=reply_markup
    )

async def handle_back_to_simplified(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    simplified = context.user_data.get('simplified_text', '').strip()
    strength = context.user_data.get('last_strength', 'medium')

    if not simplified:
        await safe_edit_message(query, "❌ Нет упрощённого текста.")
        return

    strength_names = {
        'light': 'лёгкий',
        'medium': 'средний',
        'strong': 'сильный'
    }
    strength_name = strength_names.get(strength, 'средний')

    await safe_edit_message(
        query,
        f"✅ *Упрощённый текст ({strength_name}):*\n\n{simplified}",
        reply_markup=get_simplify_keyboard(),
        parse_mode='Markdown'
    )

async def handle_simplify(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    data = query.data
    strength = data.split("_")[1]
    text = context.user_data.get('pending_text', '').strip()

    if not text:
        await safe_edit_message(query, "❌ Не найден текст для упрощения.")
        return

    await safe_edit_message(query, f"🔄 Упрощаю текст ({strength} уровень)...")

    if len(text) > 2000:
        parts = split_text(text, 1500)
        total_parts = len(parts)

        for i, part in enumerate(parts):
            if part.strip():
                progress = (i + 1) / total_parts * 100
                await safe_edit_message(
                    query,
                    f"🔄 Упрощаю текст ({strength} уровень)...\n\n"
                    f"Прогресс: {i+1}/{total_parts} частей ({progress:.0f}%)"
                )
                await asyncio.sleep(0.5)

    thinking_messages = [
        "🧠 Анализирую структуру текста...",
        "✍️ Упрощаю с сохранением смысла...",
        "🔍 Переписываю — чтобы было ясно...",
        "🧩 Готовлю результат...",
        "✅ Почти готово!"
    ]

    random.shuffle(thinking_messages)
    await send_thinking_messages(query, thinking_messages)

    try:
        simplified = simplify_long_text(
            text,
            strength=strength,
            simplify_tokenizer=simplify_tokenizer,
            simplify_model=simplify_model,
            device=device
        )

        context.user_data['simplified_text'] = simplified
        context.user_data['last_strength'] = strength

        metrics = evaluate_simplification(text, simplified)

        quality_info = (
            f"\n\n📊 *Оценка упрощения:*\n"
            f"🔤 Длина: {metrics['original_length']} → {metrics['simplified_length']} слов\n"
            f"⚖️ Сохранение смысла: {metrics['keyword_overlap_%']}%\n"
            f"💡 {metrics['quality_hint']}"
        )

        warning = ""
        if len(simplified.split()) > 300:
            warning = "\n\n⚠️ *Внимание:* текст длинный — мог быть частично обрезан."

        # Обновленная клавиатура без кнопки "Фактчекинг"
        keyboard = [
            [InlineKeyboardButton("🔤 Перевести на английский", callback_data="translate")],
            [InlineKeyboardButton("🔄 Попробовать другой уровень", callback_data="change_level")],
            [InlineKeyboardButton("📄 Показать оригинал", callback_data="show_original")]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)

        await safe_edit_message(
            query,
            f"✅ *Упрощённый текст ({strength}):*\n\n{simplified}{quality_info}{warning}",
            reply_markup=reply_markup,
            parse_mode='Markdown'
        )
    except Exception as e:
        logger.error(f"Simplification error: {e}")
        await safe_edit_message(query, f"❌ Ошибка при упрощении текста: {e}")

async def handle_translate(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    simplified = context.user_data.get('simplified_text', '').strip()

    if not simplified:
        await safe_edit_message(query, "❌ Нет текста для перевода.")
        return

    await safe_edit_message(query, "🔤 Перевожу на английский...")

    try:
        translated = translate_text(
            simplified,
            translator_tokenizer=translator_tokenizer,
            translator_model=translator_model,
            bert_tokenizer=bert_tokenizer,  # Добавляем BERT
            bert_model=bert_model,          # Добавляем BERT
            device=device
        )

        keyboard = [
            [InlineKeyboardButton("⬅️ Назад к упрощённому", callback_data="back_to_simplified")]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)

        await safe_edit_message(
            query,
            f"🇬🇧 *Перевод на английский:*\n\n{translated}",
            reply_markup=reply_markup,
            parse_mode='Markdown'
        )
    except Exception as e:
        logger.error(f"Translation error: {e}")
        await safe_edit_message(query, f"❌ Ошибка перевода: {e}")

async def handle_change_level(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query

    # Проверяем, есть ли текст для упрощения
    if not context.user_data.get('pending_text'):
        await safe_edit_message(query, "❌ Нет текста для упрощения. Отправьте текст или файл сначала.")
        return

    await query.message.reply_text(
        "🔄 *Новый уровень упрощения*\nВыбери другой вариант:",
        parse_mode='Markdown',
        reply_markup=InlineKeyboardMarkup([
            # Убрана кнопка "Лёгкий"
            [InlineKeyboardButton("⚖️ Средний", callback_data="simplify_medium")],
            [InlineKeyboardButton("🔥 Сильный", callback_data="simplify_strong")],
            [InlineKeyboardButton("🔍 Фактчекинг", callback_data="fact_checking")]
        ])
    )

async def handle_show_original(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    original = context.user_data.get('pending_text', '').strip()

    if not original:
        await query.answer("❌ Оригинальный текст не найден", show_alert=True)
        return

    try:
        await query.delete_message()
    except Exception as e:
        logger.warning(f"Error deleting message: {e}")

    max_length = 4096 - 100

    if len(original) <= max_length:
        await query.message.reply_text(
            f"📜 *Оригинальный текст:*\n\n{original}",
            parse_mode='Markdown'
        )
    else:
        parts = split_text(original, max_chars=3500)
        for i, part in enumerate(parts):
            if part.strip():
                text = f"📜 *Оригинальный текст (часть {i+1}):*\n\n{part}"
                await query.message.reply_text(text, parse_mode='Markdown')
                await asyncio.sleep(0.5)

    keyboard = [[InlineKeyboardButton("⬅️ Назад к упрощённому", callback_data="back_to_simplified")]]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await query.message.reply_text(
        "Хочешь вернуться к упрощённому тексту?",
        reply_markup=reply_markup
    )

async def button_click(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()

    data = query.data
    user_id = update.effective_user.id
    logger.info(f"User {user_id} clicked: {data}")

    if data.startswith("simplify_claim_"):
        await simplify_claim(update, context)
    elif data.startswith("change_claim_level_"):
        await change_claim_level(update, context)
    elif data.startswith("simplify_claim_medium_") or data.startswith("simplify_claim_strong_"):
        # Извлекаем индекс утверждения и уровень
        parts = data.split('_')
        strength = parts[2]
        claim_index = int(parts[3])
        await simplify_claim_with_strength(update, context, claim_index, strength)
    elif data == "back_to_uploaded_text":
        await show_last_uploaded_text(update, context)
    elif data == "back_to_fact_check":
        await fact_checking_mode(update, context)
    elif data == "fact_check_help":
        await fact_check_help(update, context)
    elif data == "fact_checking":
        await fact_checking_mode(update, context)
    elif data == "back_to_simplified":
        await handle_back_to_simplified(update, context)
    elif data.startswith("simplify_"):
        await handle_simplify(update, context)
    elif data == "translate":
        await handle_translate(update, context)
    elif data == "change_level":
        await handle_change_level(update, context)
    elif data == "show_original":
        await handle_show_original(update, context)
    elif data == "back_to_main":
        keyboard = get_main_keyboard()
        await query.message.reply_text(
            "Выбери действие:",
            reply_markup=keyboard
        )
    elif data == "help":
        await show_help(update, context)
    elif data == "restart":
        await start(update, context)
    else:
        logger.warning(f"Unknown callback data: {data}")
        await query.edit_message_text("❌ Неизвестное действие")

In [ ]:
# --- Запуск бота ---
def signal_handler(sig, frame):
    """Обработчик сигналов для корректного завершения работы"""
    print("\n🛑 Получен сигнал завершения. Останавливаем бота...")
    sys.exit(0)

async def shutdown(application):
    """Корректное завершение работы приложения"""
    print("🔄 Завершаем работу бота...")
    try:
        await application.updater.stop()
        await application.stop()
        await application.shutdown()
    except Exception as e:
        print(f"❌ Ошибка при завершении работы: {e}")

async def main():
    try:
        if not BOT_TOKEN or BOT_TOKEN == "YOUR_BOT_TOKEN_HERE":
            raise RuntimeError("❌ Неверный токен бота")

        application = Application.builder().token(BOT_TOKEN).build()

        # Добавляем обработчики
        application.add_handler(CommandHandler("start", start))
        application.add_handler(CommandHandler("help", help_command))
        application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
        application.add_handler(MessageHandler(filters.Document.ALL, handle_document))
        application.add_handler(CallbackQueryHandler(button_click))

        print("🤖 Бот запущен...")
        print("💡 Для остановки бота нажмите Ctrl+C или остановите выполнение ячейки")

        # Инициализация и запуск
        await application.initialize()
        await application.start()
        await application.updater.start_polling(drop_pending_updates=True)

        # Ожидаем сигнала завершения
        while True:
            await asyncio.sleep(1)

    except Exception as e:
        print(f"❌ Ошибка при запуске бота: {e}")
        import traceback
        traceback.print_exc()
    finally:
        if 'application' in locals():
            await shutdown(application)
        print("🛑 Бот остановлен")

if __name__ == "__main__":
    # Регистрируем обработчики сигналов
    signal.signal(signal.SIGINT, signal_handler)
    signal.signal(signal.SIGTERM, signal_handler)

    try:
        asyncio.run(main())
    except KeyboardInterrupt:
        print("\n🛑 Получен сигнал завершения. Останавливаем бота...")
    except Exception as e:
        print(f"❌ Необработанное исключение: {e}")
        import traceback
        traceback.print_exc()

🤖 Бот запущен...
💡 Для остановки бота нажмите Ctrl+C или остановите выполнение ячейки
🔄 Обработка текста разбита на 4 частей по 1500 символов
🔄 Обработка части 1/4 (1491 символов)...


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1636: UserWarning: Unfeasible length constraints: `min_length` (512) is larger than the maximum possible length (300). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


🔄 Обработка части 2/4 (1338 символов)...


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1636: UserWarning: Unfeasible length constraints: `min_length` (478) is larger than the maximum possible length (300). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


🔄 Обработка части 3/4 (1255 символов)...


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1636: UserWarning: Unfeasible length constraints: `min_length` (453) is larger than the maximum possible length (300). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


🔄 Обработка части 4/4 (746 символов)...


ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_application.py", line 1195, in process_update
    await coroutine
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_basehandler.py", line 153, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-772064011.py", line 1199, in button_click
    await handle_show_original(update, context)
  File "/tmp/ipython-input-772064011.py", line 1153, in handle_show_original
    await query.message.reply_text(text, parse_mode='Markdown')
  File "/usr/local/lib/python3.12/dist-packages/telegram/_message.py", line 1095, in reply_text
    return await self.get_bot().send_message(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_extbot.py", line 2631, in send_message
    


🛑 Получен сигнал завершения. Останавливаем бота...
🔄 Завершаем работу бота...
🛑 Бот остановлен


SystemExit: 0